In [1]:
import re
def extract_from_string(string,mode='[]'):
    if mode == '[]':
        matches = re.findall(r'\[[A-Z]\]', string)
        letters = [match.strip('[]') for match in matches]

    else:
        matches = re.findall(r'\|[A-Z]\|', string)
        letters = [match.strip('|') for match in matches]
    return letters

In [ ]:
#数据读取
import sys
import os
import torch
from datasets import Dataset,load_dataset,load_from_disk


# 获取上一级目录的路径
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, parent_dir)
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"  # 只使用 GPU 3
ds = load_dataset("rryisthebest/rank_zephyr_training_data_alpha")
train_data = ds['train']


train_list = train_data.to_list()
for index in range(len(train_list)):
    prompt =  train_list[index]['conversations'][1]['value']
    prompt = prompt.replace('The output format should be [] > [], e.g., [B] > [A]','The output format should be formulated with |?|>|?| , e.g., |C| > |B| > |A| > |D|')
    prompt = prompt.replace('each indicated by a alphabetic identifier []','each indicated by an identifier |?|')
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        prompt = prompt.replace("[%s]"%letter, f"|{letter}|") 
    train_list[index]['conversations'][1]['value'] = prompt
train_data = Dataset.from_list(train_list)
#去除特别少的<
def func_1(example): # 去除有重复元素
    extracted = extract_from_string(example['conversations'][2]['value'])
    return len(extracted) >= 20#保留ranking大于10的部分   
train_data = train_data.filter(func_1)


train_list = train_data.to_list()


In [ ]:
from tqdm import tqdm
import re
for i,batch in tqdm(enumerate(train_list)):
    label = batch['conversations'][-1]['value']
    label = label.replace('[','|')
    label = label.replace(']','|')
    train_list[i]['conversations'][-1]['value'] = label
    # if j < 2464:


In [ ]:
train_list[9]

In [ ]:
from datasets import Dataset, DatasetDict
save_data = Dataset.from_list(train_list)#.shuffle()
save_data.save_to_disk("/data/yingpeng/efficient_inference/dataset/rank_zephyr_IR_20_task/Llama-3.2-3B-Instruct")